# Dataset preparation for CAP classification 
- Authors - Shruti Murarka, Aditya Wadichar

# Read

In [1]:
import scipy.io
import numpy as np

In [3]:
from os import listdir

fnames = [i for i in listdir("DATA_seconds")]

In [4]:
path = "DATA_seconds/" + fnames[0]
temp = scipy.io.loadmat(path)
fs=int(temp['Fs'])
sec=30

In [5]:
x=np.zeros((1,fs*sec))
y=np.array(['del'])
for f in fnames:
    path = "DATA_seconds/" + f
    temp = scipy.io.loadmat(path)
    data=temp['Data_secs']
    x=np.append(x,data,axis=0)
    y=np.append(y,np.repeat(f[:3],len(data)))
x=np.delete(x,0,0)
y=np.delete(y,0,0)

In [6]:
# np.savez('DATA_seconds_tot', x=x, y=y)

# Balance

In [1]:
import numpy as np

In [2]:
arrays=np.load('DATA_seconds_tot.npz')
x=arrays['x']
y=arrays['y']

In [3]:
from collections import Counter
print(Counter(y))

Counter({'nfl': 26685, 'rbd': 20601, 'plm': 7297, 'ins': 5596, 'nar': 5307, 'nor': 4692, 'bru': 1007, 'sdb': 780})


In [10]:
mask = (y=='bru') + (y=='plm') + (y=='sdb')
print(np.sum(mask))

9084


In [11]:
from imblearn.under_sampling import RandomUnderSampler

# Reshape x for undersampling
rus = RandomUnderSampler(random_state=42)
x_resampled, y_resampled = rus.fit_resample(x[mask], y[mask])
print(Counter(y_resampled))

Counter({'bru': 780, 'plm': 780, 'sdb': 780})


In [12]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Encode the string labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_resampled)

# One-hot encode the integer labels
y_encoded = to_categorical(y_encoded)
classes=label_encoder.classes_


In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_encoded, test_size=0.2, random_state=42, stratify=y_resampled)


In [14]:
import pickle
with open('D:/Users/USER/gits/patronesproyecto/DATA_seconds_bal_xy_tt.pkl', 'wb') as f:
    pickle.dump([x_train, x_test, y_train, y_test,classes], f)